#Importing libraries

In [ ]:
import glob
import torch
import torchvision.transforms as transforms
import torchvision
import matplotlib.pyplot as plt
import numpy as np
import argparse

In [ ]:
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.1/266.1 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.6 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import wandb
from torchvision.datasets import FashionMNIST
from torchvision import datasets, models
from torchvision.datasets.utils import download_url
from torch.utils.data import DataLoader, ConcatDataset, random_split

In [ ]:
wandb.login(key='5157ae11e5d243722bc57912a56718dc8ef2f734') #Wandb login with my key

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

#Dataset Link

In [ ]:
train_data_directory = '/content/drive/MyDrive/Deeplearning/inaturalist_12K/train/'
test_data_directory = '/content/drive/MyDrive/Deeplearning/inaturalist_12K/val/'

### Importing the pretrained VGG Net16 model

In [ ]:
vggnet = torchvision.models.vgg16(weights=True)
vggnet

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth
100%|██████████| 528M/528M [00:10<00:00, 53.1MB/s]


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

### Setting device to 'cuda' if GPU is available.

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

#Layer Add

In [ ]:
def add_layer(in_features , out_feature , dim):
    vggnet.classifier[6] = torch.nn.Linear( in_features , out_feature)
    vggnet.classifier.add_module("7", torch.nn.LogSoftmax(dim))
    vggnet
    vggnet.to(device)

#Augment train data

In [ ]:
def augment_train_data(image_size , degree , mean , std , train_d):
        transformer2 = transforms.Compose([
        transforms.Resize(image_size),
        transforms.RandomHorizontalFlip(0.5),
        transforms.RandomVerticalFlip(0.02),
        transforms.RandomRotation(degree),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
        ])
        augmented_dataset = torchvision.datasets.ImageFolder(train_data_directory, transform=transformer2)
        aug_d= len(augmented_dataset)
        aug_d= 0.2 *aug_d
        augmented_dataset, _  =  random_split(augmented_dataset, [int(aug_d), len(augmented_dataset) - int(aug_d)])
        train_d = ConcatDataset([train_d, augmented_dataset])

#Data Loader

In [ ]:
def data_loader(train_d  ,train_batch_size , is_shuffle , test_batch_size ,  val_Dataset , val_batch_size):
    train_batch_load = DataLoader(
        train_d,
        train_batch_size,
        is_shuffle)
    test_batch_load = DataLoader(
        test_data_directory,
        test_batch_size,
        is_shuffle)
    val_batch_load = DataLoader(
        val_Dataset,
        val_batch_size,
        is_shuffle)
    return train_batch_load, val_batch_load, test_batch_load

# Loading the iNaturalist Dataset

In [ ]:
def load_dataset(train_batch_size, val_batch_size, test_batch_size , image_size , apply_data_augmentation ,degree , mean, std):
    transformer1 = transforms.Compose([
        transforms.Resize(image_size),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ])

    train_d = torchvision.datasets.ImageFolder(train_data_directory, transform=transformer1)
    temp =len(train_d)
    temp = 0.8* temp

    train_d, val_Dataset = random_split(train_d, [int(temp), len(train_d) - int(temp)])

    if (apply_data_augmentation):
        augment_train_data(image_size , degree , mean , std , train_d)
    return data_loader(train_d  ,train_batch_size , True , test_batch_size ,  val_Dataset , val_batch_size)


### Freezing the Model Parameters

In [ ]:
for param in vggnet.parameters():
    boolT=False
    param.requires_grad = boolT

### Adding One more layer to the model.

In [ ]:
add_layer(in_features=4096 , out_feature=10 , dim=1)

## Train

In [ ]:
def train(model, learning_rate, epochs, apply_wandb_log , image_size , apply_data_augmentation):
    mean=[0.4602, 0.4495, 0.3800]
    degree=45
    std=[0.2040, 0.1984, 0.1921]
    train_batch_load, val_batch_load, test_batch_load = load_dataset(64, 16, 16 , image_size ,apply_data_augmentation ,degree, mean , std)
    loss_function = torch.nn.CrossEntropyLoss()
    epoch=0
    optimizer = torch.optim.Adam(params=model.parameters(), lr=learning_rate, weight_decay =1e-4)
    while epoch < epochs:
        train_accuracy = 0
        train_loss = 0
        test_accuracy = 0
        test_loss = 0
        model.train()
        for i, (images, labels) in enumerate(train_batch_load):

            images= images.to(device)
            labels = labels.to(device)
            optimizer.zero_grad()   # doing zero gradient.
            y_pred = model(images) #forward Propagation
            loss = loss_function(y_pred, labels) # Calculating Loss.
            loss.backward() # Backward Propagation
            optimizer.step() # update rule

            train_loss =train_loss +  loss.item()

            _, prediction = torch.max(y_pred.data, 1)
            train_accuracy =train_accuracy +  int(torch.sum(prediction == labels.data))
        count_train = len(glob.glob(train_data_directory+'/**/*.jpg'))
        train_accuracy = train_accuracy / count_train
        train_loss = train_loss / count_train
        print(f"Epochs : {epoch+1} Train Accuracy : {train_accuracy / count_train} Train Loss {train_loss}")


        with torch.no_grad():
            model.eval()
            for i, (images, labels) in enumerate(val_batch_load):
                images = images.to(device)
                labels = labels.to(device)

                y_pred = model(images)

                loss = loss_function(y_pred, labels)
                test_loss = test_loss + loss.item()

                _, predicted = torch.max(y_pred.data, 1)

                test_accuracy =  test_accuracy  + int(torch.sum(predicted == labels.data))
            count_test = len(glob.glob(test_data_directory+'/**/*.jpg'))
            test_accuracy =test_accuracy / count_test
            test_loss =test_loss / count_test

            print(f"Epochs : {epoch+1} Validation Accuracy : {test_accuracy} Validation Loss {test_loss}")
            if(apply_wandb_log):
                wandb.log({"train_accuracy": train_accuracy, "train_loss" : train_loss, "val_accuracy": test_accuracy, "val_error": test_loss})
        epoch+=1

## Start Training

In [ ]:
def sweep_(learning_rate ,apply_wandb_log ,epochs , image_size, apply_data_augmentation):
    run = wandb.init( project='DL2')
    run.name = 'ep_' + str(epochs) +  '_lr_'+ str(learning_rate) + '_nf_'
    train(vggnet, learning_rate, epochs,apply_wandb_log  , image_size , apply_data_augmentation)

In [ ]:
sweep_(learning_rate = 0.001 ,apply_wandb_log=True ,  epochs = 10 , image_size=(256,256) ,apply_data_augmentation=True)

wandb: Currently logged in as: cs23m013. Use `wandb login --relogin` to force relogin


Epochs : 1 Train Accuracy : 0.005 Train Loss 0.11500859260559082
Epochs : 1 Validation Accuracy : 0.0 Validation Loss 0.11035106182098389
Epochs : 2 Train Accuracy : 0.0125 Train Loss 0.0986224889755249
Epochs : 2 Validation Accuracy : 0.0 Validation Loss 0.11015198230743409
